# Solving Frozen Lake Problem Using Value Iteration

Frozen lake involves crossing a frozen lake from Start(S) to Goal(G) without falling into any Holes(H) by walking over the Frozen(F) lake.  
The agent may not always move in the intended direction due to the slippery nature of the frozen lake. It will move in the intended direction with probability of 1/3 else will move in either perpendicular direction with equal probability of 1/3 in both directions.  
For example, if action is left, then:  
        - P(move left)=1/3  
        - P(move up)=1/3  
        - P(move down)=1/3  


### Action Space  

The agent takes a 1-element vector for actions. The action space is `(dir)`, where `dir` decides direction to move in which can be:  
    - 0: LEFT  
    - 1: DOWN  
    - 2: RIGHT  
    - 3: UP  
    
 ### Observation Space  
 
The observation is a value representing the agent's current position as current_row * nrows + current_col (where both the row and col start at 0). For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15.
The number of possible observations is dependent on the size of the map.  
For example, the 4x4 map has 16 possible observations.  


### Rewards  
  
Reward schedule:  
    - Reach goal(G): +1  
    - Reach hole(H): 0  
    - Reach frozen(F): 0  
       

In [ ]:
import gym
import numpy as np

Initialize the Frozen Lake environment:

In [ ]:
env = gym.make('FrozenLake-v1')

 Let's take a look at how the environment looks like:

In [ ]:
env.render()


SFFF
FHFH
FFFH
HFFG


Our agent is in the state S and the goal is to reach state G without visiting the H states.

Now we define the function called value_iteration which performs value iteration and returns the optimal value
function (value table).

In [ ]:
gamma=1.0

def value_iteration(env):
    
    # initialize value table with zeros
    value_table = np.zeros(env.observation_space.n)
    
    # set number of iterations and threshold
    num_iterations = 100000
    threshold = 1e-22
    
    for i in range(num_iterations):
        
        # On each iteration, copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table) 
        
        # Now we calculate Q Value for each actions in the state 
        # and update the value of a state with maximum Q value        
        for state in range(env.observation_space.n):
            Q_values = [sum([trans_prob*(reward + gamma*updated_value_table[next_state])
                             for trans_prob, next_state, reward, _ in env.P[state][action]]) 
                                   for action in range(env.action_space.n)] 
                                                        
            value_table[state] = max(Q_values)            
        # we check whether we have reached the convergence, that is, whether the difference 
        # between our value table and updated value table is less than a threshold value. 
        # If it is, we break the loop and return the value function as optimal value function
        
        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
             print ('Value-iteration converged at iteration #', i+1)
             break
    
    return value_table

Now, we define a function called extract_policy for extracting optimal policy from the optimal value function.  
We calculate Q value using our optimal value function and pick up
the actions which has the highest Q value for each state as the optimal policy.

In [ ]:
def extract_policy(value_table):
 
    # initialize the policy with zeros
    policy = np.zeros(env.observation_space.n)     
    
    for state in range(env.observation_space.n):
                
        # compute Q value for all ations in the state
        for action in range(env.action_space.n):
            Q_values = [sum([trans_prob*(reward + gamma*value_table[next_state])
                             for trans_prob, next_state, reward, _ in env.P[state][action]]) 
                                   for action in range(env.action_space.n)] 
                
        #extract policy by selecting the action which has maximum Q value
        policy[state] = np.argmax(np.array(Q_values))        
    
    return policy

First, We compute the optimal value function

In [ ]:
optimal_value_function = value_iteration(env=env)

Value-iteration converged at iteration # 1373


and we derive the optimal policy from the optimal value function

In [ ]:
optimal_policy = extract_policy(optimal_value_function)

In [ ]:
print(optimal_policy)

[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


In [ ]:
env.render()


SFFF
FHFH
FFFH
HFFG


Best policy for each state:  

[0. 3. 3. 3.   
 0. 0. 0. 0.   
 3. 1. 0. 0.   
 0. 2. 1. 0.]  

If you are in state 1, for example, the best policy is 3 (go up). This will make it move in the intended direction with probability of 1/3, else it will move in any perpendicular direction (left or right) with equal probability of 1/3 in both directions. Which means that with this policy the agent will not fall into the hole below.
  